In [33]:
import sys, os
from os.path import join
import tomopy
import pydicom
from pydicom.data import get_testdata_files
import numpy as np
import scipy
from skimage.transform import radon, iradon

## Data Divisions

### Train
Contains all .dcm files from the VHF directories (Pelvis, Head, Hip), and nothing else. Should be no other directories.

### Test
Contains all .dcm files from the S70 directories, and nothing else.

Note: mv0 data is thrown out as it couldn't be read.

In [36]:
curr_dir = os.getcwd()
data_dir = join(curr_dir, 'data')
train_dir = join(data_dir, 'train')
test_dir = join(data_dir, 'test')

train_gt_dir = join(train_dir, 'gt') # Store ground truth
train_fbp_dir = join(train_dir, 'fbp') # Store filtered backprojctions
train_raw_dir = join(train_dir, 'raw') # Store 'raw' data
test_gt_dir = join(test_dir, 'gt') # Store ground truth
test_fbp_dir = join(test_dir, 'fbp') # Store filtered backprojections
test_raw_dir = join(test_dir, 'raw') # Store 'raw' data

# Add .dcm to all the files in test, since they aren't present
# Note: only do this once!!!
for filename in os.listdir(test_dir):
    if filename.endswith(''):
        path = join(test_dir, filename)
        newpath = path + '.dcm'
        os.rename(path,newpath)


if not os.path.isdir(train_gt_dir): os.mkdir(train_gt_dir)
if not os.path.isdir(train_fbp_dir): os.mkdir(train_fbp_dir)
if not os.path.isdir(train_raw_dir): os.mkdir(train_raw_dir)
if not os.path.isdir(test_gt_dir): os.mkdir(test_gt_dir)
if not os.path.isdir(test_fbp_dir): os.mkdir(test_fbp_dir)
if not os.path.isdir(test_raw_dir): os.mkdir(test_raw_dir)


In [ ]:
# Go through files and save CT data only into the ground truth folders
for filename in os.listdir(train_dir):
    if filename.endswith('.dcm'):
        file_id = filename.split('.')[1] # id we will use to save data
        path = join(train_dir,filename) # full path of the file
        ds = pydicom.dcmread(path)
        
        # Extract ct data (ground truth) and save
        ct = ds.pixel_array
        save_name = file_id + '_gt'
        save_path = join(train_gt_dir, save_name)
        np.save(save_path, ct)
        
        # Perform radon trasnform for 'raw' data
        theta = np.linspace(0, 180, max(ct.shape)/2, endpoint = False) # for a worse reconstruction
        sino = radon(ct, theta=theta)
        save_name = file_id + '_raw'
        save_path = join(train_raw_dir, save_name)
        np.save(save_path, sino)
        
        # Perform fbp using iradon (not a great method but it's okay)
        fbp = iradon(sino, theta=theta)
        save_name = file_id + 'fbp'
        save_path = join(train_fbp_dir, save_name)
        np.save(save_path, fbp)

C:\Users\laian\Anaconda\envs\py37\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  from ipykernel import kernelapp as app
C:\Users\laian\Anaconda\envs\py37\lib\site-packages\skimage\transform\radon_transform.py:204: UserWarning: The default of `circle` in `skimage.transform.iradon` will change to `True` in version 0.15.
  warn('The default of `circle` in `skimage.transform.iradon` '


In [ ]:
for filename in os.listdir(test_dir):
    if filename.endswith('.dcm'):
        file_id = filename.split('.')[0] # id we will use to save data
        path = join(test_dir,filename) # full path of the file
        ds = pydicom.dcmread(path)
        
        # Extract ct data (ground truth) and save
        ct = ds.pixel_array
        save_name = file_id + '_gt'
        save_path = join(test_gt_dir, save_name)
        np.save(save_path, ct)
        
        # Perform radon trasnform for 'raw' data
        theta = np.linspace(0, 180, max(ct.shape)/2, endpoint = False) # for a worse reconstruction
        sino = radon(ct, theta=theta)
        save_name = file_id + '_raw'
        save_path = join(test_raw_dir, save_name)
        np.save(save_path, sino)
        
        # Perform fbp using iradon (not a great method but it's okay)
        fbp = iradon(sino, theta=theta)
        save_name = file_id + 'fbp'
        save_path = join(test_fbp_dir, save_name)
        np.save(save_path, fbp)